<a href="https://colab.research.google.com/github/AggelosRDio/MscMachineLearning/blob/master/ML3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 3

## Library Imports

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime, timedelta
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, StackingRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

## Download Financial Data

In [2]:
ticker_symbol = "DX-Y.NYB"
ticker = yf.Ticker(ticker_symbol)
df = ticker.history(period="5y").reset_index(drop = False)
df = df.drop(columns=['Volume', 'Dividends', 'Stock Splits'])
df

,Date,Open,High,Low,Close
0,2020-11-05 00:00:00-05:00,93.339996,93.550003,92.489998,92.529999
1,2020-11-06 00:00:00-05:00,92.570000,92.820000,92.180000,92.230003
2,2020-11-09 00:00:00-05:00,92.239998,92.959999,92.129997,92.730003
3,2020-11-10 00:00:00-05:00,92.760002,92.970001,92.599998,92.750000
4,2020-11-11 00:00:00-05:00,92.699997,93.209999,92.610001,93.040001
...,...,...,...,...,...
1253,2025-10-30 00:00:00-04:00,99.139999,99.720001,98.919998,99.529999
1254,2025-10-31 00:00:00-04:00,99.489998,99.839996,99.419998,99.800003
1255,2025-11-03 00:00:00-05:00,99.750000,99.989998,99.709999,99.870003
1256,2025-11-04 00:00:00-05:00,99.980003,100.260002,99.739998,100.220001


## Question 1

In [3]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = df["Date"], y=df["Open"], mode="lines", name="Open", line=dict(color="blue")))
fig.add_trace(go.Scatter(x = df["Date"], y=df["Close"], mode="lines", name="Close", line=dict(color="orange")))
fig.add_trace(go.Scatter(x = df["Date"], y=df["High"], mode="lines", name="High", line=dict(color="green")))
fig.add_trace(go.Scatter(x = df["Date"], y=df["Low"], mode="lines", name="Low", line=dict(color="red")))
fig.show()

## Question 2

In [4]:
df_rolling = df.copy()
df_rolling["Rolling Mean"] = df["Close"].rolling(window=30).mean()
df_rolling["Rolling Std"] = df["Close"].rolling(window=30).std()


fig2 = go.Figure()
fig2.add_trace(go.Scatter(x = df.index, y=df_rolling["Close"], mode="lines", name="Close", line=dict(color="blue")))
fig2.add_trace(go.Scatter(x = df.index, y=df_rolling["Rolling Mean"], mode="lines", name=f"30-day rolling mean", line=dict(color="orange")))
fig2.add_trace(go.Scatter(x = df.index, y=df_rolling["Rolling Std"], mode="lines", name="30-day rolling Std", line=dict(color="red")))
fig2.show()

We can observe that the rolling mean and rolling standard deviation both show variance in time, which indicates that the timeseries is not stationary.

## Question 3

By training on non-stationary time series the model may find false correlations caused by trends, it may learn on patterns that apply only to a specific time period and it may generate predictions that will be unreliable if a trend or variance changes in the future.

In our specific data, we see large price flunctuations and if we train a model on an upward trend, it will fail during a downward trend.

## Question 4

In [5]:
df_summer_2025 = df[(df['Date'] >= '2025-06-01') & (df['Date'] < '2025-08-01')]
start = df_summer_2025['Close'].iloc[0]
end = df_summer_2025['Close'].iloc[-1]
fig3 = go.Figure()
fig3.add_trace(go.Scatter(x = df_summer_2025['Date'], y=df_summer_2025["Close"], mode="lines", name="Close", line=dict(color="blue")))


The dollar is in a downward trend from start of June all the way to early July, after which it seems to be going upwards again. This can be explained by a number of factors:



1. On June 13th begun the Israel-Iran war that lasted until June 25th. Due to the geopolitical risk, currency dynamics (and USD in particular) has been affected (source: https://en.wikipedia.org/wiki/Iran%E2%80%93Israel_war)
2. Suprisingly strong US job data seem to have been able to halt the downward trend and have it start rising again (source: https://www.theguardian.com/business/live/2025/jul/03/pound-bonds-calm-sell-off-starmer-backs-reeves-uk-economy-us-jobs-ftse-business-live-news-updates)



## Question 5

It is possible to make a prediction of next day's data, however, not with good precision.

The reason for this is that the data of a single day do not contain enough information to predict the future. All available information is incorporated to the price of the current day and tomorrow's data are dependend on information not known today (Efficient-market hypothesis: https://en.wikipedia.org/wiki/Efficient-market_hypothesis)

It is possible however to approxiate a function

**`f(oₜ, hₜ, lₜ, cₜ) ≈ cₜ₊₁`**

This takes advantage of patterns and trends, seasonality and historical timeframes (not just a single day).

For this reason it is possible to use timeframes with N > 1 days.

## Question 6

It doesn't make sense to do random split in train and test sets, because in timeseries the chronological order is integral in how next values are calculated. Splitting randomly would potentially cause future values to be used to predict past values, which doesn't make sense in practice.

## Question 7

In [6]:
df_train = df[(df['Date'] < '2024-01-01')]
df_test = df[(df['Date'] >= '2024-01-01') & (df['Date'] < '2025-01-01')]

train_dates = df_train['Date']
test_dates = df_test['Date']

df_train = df_train.drop(columns=['Date'])
df_test = df_test.drop(columns=['Date'])
print(df_train.columns.to_list())
print(df_test.columns.to_list())


['Open', 'High', 'Low', 'Close']
['Open', 'High', 'Low', 'Close']


## Question 8

In [7]:
def create_timeframes(df, N):
  x, y = [], []

  for i in range(len(df) - N):
    x.append(df.iloc[i: i + N][['Open', 'High', 'Low', 'Close']].values)
    y.append(df.iloc[i + N]['Close'])

  return np.array(x), np.array(y)


## Question 9

To predict the next day's data, recent prices are much more important. Large N would introduce noise from distant data that may no longer be relevant, so N should be kept small.

For long term predictions we need to understand patterns and trends. As such a larger sample helps identify seasonal patterns and as such N should be large.

## Question 10

In [8]:
x_train, y_train = create_timeframes(df_train, 5)
x_test, y_test = create_timeframes(df_test, 5)

x_train = x_train.reshape(x_train.shape[0], -1)
x_test = x_test.reshape(x_test.shape[0], -1)

print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print("")
print(f"x_train first item: ")
print(x_train[0])

x_train shape: (787, 20)
y_train shape: (787,)

x_train first item: 
[93.33999634 93.55000305 92.48999786 92.52999878 92.56999969 92.81999969
 92.18000031 92.23000336 92.23999786 92.95999908 92.12999725 92.73000336
 92.76000214 92.97000122 92.59999847 92.75       92.69999695 93.20999908
 92.61000061 93.04000092]


## Question 11

In [9]:
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(),
    "Bagging": BaggingRegressor(),
    "XGBoost": XGBRegressor(),
    "Stacking": StackingRegressor(
        estimators=[
            ('lr', LinearRegression()),
            ('dt', DecisionTreeRegressor())
        ],
        final_estimator=LinearRegression(),
        n_jobs=-1
    )
}

results = {}

for model_name, model in models.items():

  model.fit(x_train, y_train)

  y_train_pred = model.predict(x_train)
  y_test_pred = model.predict(x_test)

  train_mae = mean_absolute_error(y_train, y_train_pred)
  test_mae = mean_absolute_error(y_test, y_test_pred)

  results[model_name] = {
      "Model": model,
      "Train MAE": train_mae,
      "Test MAE": test_mae,
      "Y Test Prediction": y_test_pred
  }

fig4 = go.Figure(data=[
    go.Bar(name="Train MAE", x=list(results.keys()), y=[result["Train MAE"] for result in results.values()], marker_color="blue"),
    go.Bar(name="Test MAE", x=list(results.keys()), y=[result["Test MAE"] for result in results.values()], marker_color="green")
])

fig4.update_layout(
    title="",
    xaxis_title="Model",
    yaxis_title="MAE",
    barmode="group"
)

fig4.show()


1. Linear regression finds the linear relationship "y = b + w1x1 + w2x2 +... " that minimizes the squared error. Though simple and fast, it assumes linearity in the data.
2. Random forest is an ensemble of many decision trees, where each tree is trained on a random subset of the data and features and the final prediction is the average of all the trees.
3. Bagging trains multiple linear regression models on random subsets of the training set (with replacement) and averages their predictions to reduce variance.
4. XGBoost trains trees sequentially and each new tree attempts to correct the errors of the previous one.
5. Stacking uses linear regression and decision tree as base models and a meta-model learns how to combine their predictions.

## Question 12

In [10]:
df_dates = df.copy()
df_dates['Month'] = pd.to_datetime(df_dates['Date']).dt.month
df_dates['Day'] = pd.to_datetime(df_dates['Date']).dt.day

df_dates

,Date,Open,High,Low,Close,Month,Day
0,2020-11-05 00:00:00-05:00,93.339996,93.550003,92.489998,92.529999,11,5
1,2020-11-06 00:00:00-05:00,92.570000,92.820000,92.180000,92.230003,11,6
2,2020-11-09 00:00:00-05:00,92.239998,92.959999,92.129997,92.730003,11,9
3,2020-11-10 00:00:00-05:00,92.760002,92.970001,92.599998,92.750000,11,10
4,2020-11-11 00:00:00-05:00,92.699997,93.209999,92.610001,93.040001,11,11
...,...,...,...,...,...,...,...
1253,2025-10-30 00:00:00-04:00,99.139999,99.720001,98.919998,99.529999,10,30
1254,2025-10-31 00:00:00-04:00,99.489998,99.839996,99.419998,99.800003,10,31
1255,2025-11-03 00:00:00-05:00,99.750000,99.989998,99.709999,99.870003,11,3
1256,2025-11-04 00:00:00-05:00,99.980003,100.260002,99.739998,100.220001,11,4


Month can improve predictions because it introduces seasonality. Exchange rates can be affected by season patterns, such as the tourist season.

In [11]:
columns = ['Open', "High", "Low", "Close"]

for col in columns:
  df_dates[col] = np.log(df_dates[col] / df_dates[col].shift(1))

df_log = df_dates.dropna().reset_index(drop=True)

df_log

,Date,Open,High,Low,Close,Month,Day
0,2020-11-06 00:00:00-05:00,-0.008284,-0.007834,-0.003357,-0.003247,11,6
1,2020-11-09 00:00:00-05:00,-0.003571,0.001507,-0.000543,0.005407,11,9
2,2020-11-10 00:00:00-05:00,0.005622,0.000108,0.005089,0.000216,11,10
3,2020-11-11 00:00:00-05:00,-0.000647,0.002578,0.000108,0.003122,11,11
4,2020-11-12 00:00:00-05:00,0.003339,-0.000751,0.001834,-0.000860,11,12
...,...,...,...,...,...,...,...
1252,2025-10-30 00:00:00-04:00,0.004043,0.003617,0.003037,0.003119,10,30
1253,2025-10-31 00:00:00-04:00,0.003524,0.001203,0.005042,0.002709,10,31
1254,2025-11-03 00:00:00-05:00,0.002610,0.001501,0.002913,0.000701,11,3
1255,2025-11-04 00:00:00-05:00,0.002303,0.002697,0.000301,0.003498,11,4


In [12]:
fig5 = go.Figure()

for col in columns:
  fig5.add_trace(go.Histogram(x=df_log[col], name=col, opacity=0.7, nbinsx=50))
fig5.show()

Log returns are more stationary than absolute prices and their mean and variance remain stable over time. Furthermore they approximate a normal distribution, which benefits many ML models that assume normality.

If a model predicts C(t+1) = 0.01 in log returns, then:

LogReturn = C(t+1) = ln(Price(t+1)/Price(t)) = 0.01 <=>

Price(t+1) / Price(t) = e^0.01 ≈ 1.01

This means an increase of approximately 1% in the closing price.

<br/>

---

<br/>

Assuming C(t) = 0.95$ then:

Price(t+1) = Price(t) * e^0.01
Price(t+1) = 0.95 * 1.01
Price(t+1) = 0.9595$



In [13]:
c_t = 0.95
prediction = 0.01
c_t_1 = c_t * np.exp(prediction)
print(c_t_1)

0.9595476587299595


### Rerunning Previous Steps

In [14]:
df_train_new = df_log[(df_log['Date'] < '2024-01-01')]
df_test_new = df_log[(df_log['Date'] >= '2024-01-01') & (df_log['Date'] < '2025-01-01')]

train_dates_new = df_train_new['Date']
test_dates_new = df_test_new['Date']

df_train_new = df_train_new.drop(columns=['Date'])
df_test_new = df_test_new.drop(columns=['Date'])

x_train_new, y_train_new = create_timeframes(df_train_new, 5)
x_test_new, y_test_new = create_timeframes(df_test_new, 5)

x_train_new = x_train_new.reshape(x_train_new.shape[0], -1)
x_test_new = x_test_new.reshape(x_test_new.shape[0], -1)

print(f"x_train shape: {x_train_new.shape}")
print(f"y_train shape: {y_train_new.shape}")

models_new = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(),
    "Bagging": BaggingRegressor(),
    "XGBoost": XGBRegressor(),
    "Stacking": StackingRegressor(
        estimators=[
            ('lr', LinearRegression()),
            ('dt', DecisionTreeRegressor())
        ],
        final_estimator=LinearRegression(),
        n_jobs=-1
    )
}

results_new = {}
for name, model in models_new.items():

    model.fit(x_train_new, y_train_new)

    y_train_pred_new = model.predict(x_train_new)
    y_test_pred_new = model.predict(x_test_new)

    train_mae_new = mean_absolute_error(y_train_new, y_train_pred_new)
    test_mae_new = mean_absolute_error(y_test_new, y_test_pred_new)

    results_new[name] = {
        "Model": model,
        "Train MAE": train_mae_new,
        "Test MAE": test_mae_new,
        "y Test Prediction": y_test_pred_new
    }

model_names_new = list(results_new.keys())
train_mae_values_new = [results_new[name]["Train MAE"] for name in model_names_new]
test_mae_values_new = [results_new[name]["Test MAE"] for name in model_names_new]

fig6 = go.Figure(data=[
    go.Bar(name="Train MAE", x=model_names_new, y=train_mae_values_new, marker_color="blue"),
    go.Bar(name="Test MAE", x=model_names_new, y=test_mae_values_new, marker_color="green")
])

fig6.show()


x_train shape: (786, 20)
y_train shape: (786,)


### Previous and Current Comparison

In [15]:
comparison_df = pd.DataFrame({
    'Model': model_names_new,
    'Previous MAE': [results[name]["Test MAE"] for name in list(results.keys())],
    'Current MAE': [results_new[name]["Test MAE"] for name in model_names_new]
})

comparison_df["Improvement (%)"] = ((comparison_df["Previous MAE"] - comparison_df["Current MAE"]) / comparison_df["Previous MAE"] * 100)

comparison_df

,Model,Previous MAE,Current MAE,Improvement (%)
0,Linear Regression,0.284580,0.002781,99.022898
1,Random Forest,0.321779,0.002791,99.132649
2,Bagging,0.375891,0.003083,99.179861
3,XGBoost,0.380252,0.003081,99.189616
4,Stacking,0.287638,0.002715,99.056100


## Question 13

In [16]:
best_model_name = min(results_new, key=lambda x: results_new[x]["Test MAE"])
best_model_results = results_new[best_model_name]

print(f"best model: {best_model_name}")
print(f"Test MAE: {best_model_results["Test MAE"]}")

y_pred_log = best_model_results["y Test Prediction"]
N = 5
df_test_original = df[(df['Date'] >= '2024-01-01') & (df['Date'] < '2025-01-01')]
close_values = df_test_original['Close'].iloc[N-1:N-1+len(y_test_new)].values

y_pred_abs = close_values * np.exp(y_pred_log)
y_true_abs = close_values * np.exp(y_test_new)

dates = df_test_original['Date'].iloc[N:N+len(y_test_new)].values

fig7 = go.Figure()
fig7.add_trace(go.Scatter(x=dates, y=y_true_abs, mode='lines', name='True Values'))
fig7.add_trace(go.Scatter(x=dates, y=y_pred_abs, mode='lines', name='Predicted Values'))

fig7.show()


best model: Stacking
Test MAE: 0.0027150129010824033
